# 📊 Phân tích Dữ liệu Mua sắm Trực tuyến - HUIT Big Data

**Tác giả:** Sinh viên HUIT  
**Môn học:** Big Data Analytics  
**Mục tiêu:** Phân tích hành vi khách hàng và xây dựng hệ thống đề xuất sản phẩm

## 🎯 Mục tiêu Phân tích

1. **Exploratory Data Analysis (EDA)** - Khám phá dữ liệu
2. **Customer Segmentation** - Phân khúc khách hàng
3. **Product Analysis** - Phân tích sản phẩm
4. **Recommendation System** - Hệ thống đề xuất
5. **Business Insights** - Thông tin kinh doanh

## 📦 Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Configure plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Vietnamese font support (if available)
try:
    plt.rcParams['font.family'] = ['Arial Unicode MS', 'Arial', 'sans-serif']
except:
    print("Vietnamese font not available, using default")

print("📚 Libraries imported successfully!")

## 📁 Load Data

In [ ]:
# Load datasets
try:
    customers_df = pd.read_csv('../data/sample/customers.csv')
    products_df = pd.read_csv('../data/sample/products.csv')
    transactions_df = pd.read_csv('../data/sample/transactions.csv')
    
    print("✅ Data loaded successfully!")
    print(f"📊 Customers: {len(customers_df):,} records")
    print(f"📦 Products: {len(products_df):,} records")
    print(f"💳 Transactions: {len(transactions_df):,} records")
    
except FileNotFoundError:
    print("❌ Data files not found!")
    print("Please run the data generation script first:")
    print("python data/sample/generate_data.py")
    
    # Generate sample data if not exists
    import sys
    sys.path.append('../')
    
    from data.sample.generate_data import DataGenerator
    
    print("🔄 Generating sample data...")
    generator = DataGenerator()
    
    customers_df = generator.generate_customers(1000)
    products_df = generator.generate_products(500)
    transactions_df = generator.generate_transactions(customers_df, products_df, 10000)
    
    # Save data
    generator.save_data(customers_df, products_df, transactions_df, '../data/sample')
    print("✅ Sample data generated and saved!")

## 🔍 Exploratory Data Analysis (EDA)

### Dataset Overview

In [ ]:
# Dataset info
print("🔍 DATASET OVERVIEW")
print("=" * 50)

print("\n📊 CUSTOMERS")
print(customers_df.info())
print("\nFirst 5 records:")
display(customers_df.head())

print("\n📦 PRODUCTS")
print(products_df.info())
print("\nFirst 5 records:")
display(products_df.head())

print("\n💳 TRANSACTIONS")
print(transactions_df.info())
print("\nFirst 5 records:")
display(transactions_df.head())

In [ ]:
# Data preprocessing
transactions_df['timestamp'] = pd.to_datetime(transactions_df['timestamp'])
transactions_df['date'] = transactions_df['timestamp'].dt.date
transactions_df['hour'] = transactions_df['timestamp'].dt.hour
transactions_df['day_of_week'] = transactions_df['timestamp'].dt.day_name()
transactions_df['month'] = transactions_df['timestamp'].dt.month_name()

print("✅ Data preprocessing completed!")

### Basic Statistics

In [ ]:
# Basic statistics
print("📈 BASIC STATISTICS")
print("=" * 50)

total_revenue = transactions_df['total_amount'].sum()
avg_order_value = transactions_df['total_amount'].mean()
total_customers = customers_df['customer_id'].nunique()
total_products = products_df['product_id'].nunique()
active_customers = transactions_df['customer_id'].nunique()

stats_data = {
    'Metric': [
        'Total Revenue (VND)',
        'Average Order Value (VND)',
        'Total Customers',
        'Active Customers', 
        'Total Products',
        'Total Transactions',
        'Customer Activation Rate (%)'
    ],
    'Value': [
        f"{total_revenue:,.0f}",
        f"{avg_order_value:,.0f}",
        f"{total_customers:,}",
        f"{active_customers:,}",
        f"{total_products:,}",
        f"{len(transactions_df):,}",
        f"{(active_customers/total_customers*100):.1f}%"
    ]
}

stats_df = pd.DataFrame(stats_data)
display(stats_df)

## 📊 Data Visualization

### Sales Analysis

In [ ]:
# Sales by category
category_sales = transactions_df.groupby('category').agg({
    'total_amount': 'sum',
    'quantity': 'sum',
    'transaction_id': 'count'
}).round(2)

category_sales.columns = ['Revenue', 'Quantity_Sold', 'Transactions']
category_sales = category_sales.sort_values('Revenue', ascending=False)

# Create subplots
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Revenue by Category', 'Quantity Sold by Category', 
                   'Transactions by Category', 'Category Performance'),
    specs=[[{"type": "bar"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "scatter"}]]
)

# Revenue by category
fig.add_trace(
    go.Bar(x=category_sales.index, y=category_sales['Revenue'], 
           name='Revenue', marker_color='lightblue'),
    row=1, col=1
)

# Quantity by category
fig.add_trace(
    go.Bar(x=category_sales.index, y=category_sales['Quantity_Sold'], 
           name='Quantity', marker_color='lightgreen'),
    row=1, col=2
)

# Transactions by category
fig.add_trace(
    go.Bar(x=category_sales.index, y=category_sales['Transactions'], 
           name='Transactions', marker_color='lightcoral'),
    row=2, col=1
)

# Scatter plot: Revenue vs Transactions
fig.add_trace(
    go.Scatter(x=category_sales['Transactions'], y=category_sales['Revenue'],
               mode='markers+text', text=category_sales.index,
               textposition='top center', name='Performance',
               marker=dict(size=10, color='purple')),
    row=2, col=2
)

fig.update_layout(height=800, title_text="📊 Sales Analysis by Category")
fig.show()

print("\n📊 Top 5 Categories by Revenue:")
display(category_sales.head())

In [ ]:
# Time series analysis
daily_sales = transactions_df.groupby('date')['total_amount'].sum().reset_index()
daily_sales['date'] = pd.to_datetime(daily_sales['date'])

# Sales trend
fig = px.line(daily_sales, x='date', y='total_amount',
              title='📈 Daily Sales Trend',
              labels={'total_amount': 'Revenue (VND)', 'date': 'Date'})

fig.update_layout(height=400)
fig.show()

# Sales by hour
hourly_sales = transactions_df.groupby('hour')['total_amount'].sum()

fig = px.bar(x=hourly_sales.index, y=hourly_sales.values,
             title='⏰ Sales by Hour of Day',
             labels={'x': 'Hour', 'y': 'Revenue (VND)'})

fig.update_layout(height=400)
fig.show()

### Customer Analysis

In [ ]:
# Customer demographics
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Age Distribution', 'Gender Distribution', 
                   'City Distribution', 'Registration Trend'),
    specs=[[{"type": "histogram"}, {"type": "pie"}],
           [{"type": "bar"}, {"type": "bar"}]]
)

# Age distribution
fig.add_trace(
    go.Histogram(x=customers_df['age'], nbinsx=20, name='Age'),
    row=1, col=1
)

# Gender distribution
gender_counts = customers_df['gender'].value_counts()
fig.add_trace(
    go.Pie(labels=gender_counts.index, values=gender_counts.values, name='Gender'),
    row=1, col=2
)

# City distribution (top 10)
city_counts = customers_df['city'].value_counts().head(10)
fig.add_trace(
    go.Bar(x=city_counts.index, y=city_counts.values, name='City'),
    row=2, col=1
)

# Registration trend by month
customers_df['registration_date'] = pd.to_datetime(customers_df['registration_date'])
reg_trend = customers_df['registration_date'].dt.to_period('M').value_counts().sort_index()

fig.add_trace(
    go.Bar(x=[str(x) for x in reg_trend.index], y=reg_trend.values, name='Registrations'),
    row=2, col=2
)

fig.update_layout(height=800, title_text="👥 Customer Demographics Analysis")
fig.show()

In [ ]:
# Customer behavior analysis
customer_behavior = transactions_df.groupby('customer_id').agg({
    'total_amount': ['sum', 'mean', 'count'],
    'product_id': 'nunique',
    'timestamp': ['min', 'max']
}).round(2)

# Flatten column names
customer_behavior.columns = ['total_spent', 'avg_order_value', 'num_orders', 
                           'unique_products', 'first_order', 'last_order']

# Calculate customer lifetime (days)
customer_behavior['lifetime_days'] = (
    customer_behavior['last_order'] - customer_behavior['first_order']
).dt.days

# Customer value segments
customer_behavior['value_segment'] = pd.cut(
    customer_behavior['total_spent'],
    bins=[0, 1000000, 5000000, 10000000, float('inf')],
    labels=['Low Value', 'Medium Value', 'High Value', 'VIP']
)

# Display statistics
print("💳 CUSTOMER BEHAVIOR STATISTICS")
print("=" * 50)
display(customer_behavior.describe())

# Value segment distribution
segment_dist = customer_behavior['value_segment'].value_counts()

fig = px.pie(values=segment_dist.values, names=segment_dist.index,
             title='🎯 Customer Value Segments')
fig.show()

print("\n🎯 Customer Segments:")
print(segment_dist)

## 🎯 Customer Segmentation (RFM Analysis)

RFM Analysis giúp phân khúc khách hàng dựa trên:
- **Recency (R)**: Thời gian từ lần mua cuối
- **Frequency (F)**: Tần suất mua hàng
- **Monetary (M)**: Giá trị tiền tệ

In [ ]:
# RFM Analysis
from datetime import datetime

# Calculate RFM metrics
current_date = transactions_df['timestamp'].max() + pd.Timedelta(days=1)

rfm = transactions_df.groupby('customer_id').agg({
    'timestamp': lambda x: (current_date - x.max()).days,  # Recency
    'transaction_id': 'count',  # Frequency
    'total_amount': 'sum'  # Monetary
}).round(2)

rfm.columns = ['Recency', 'Frequency', 'Monetary']

# Create RFM scores (1-5 scale)
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5,4,3,2,1])  # Lower recency = higher score
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5])
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 5, labels=[1,2,3,4,5])

# Combine RFM scores
rfm['RFM_Score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)
rfm['RFM_Total'] = rfm['R_Score'].astype(int) + rfm['F_Score'].astype(int) + rfm['M_Score'].astype(int)

# Define customer segments based on RFM scores
def rfm_segment(row):
    if row['RFM_Total'] >= 13:
        return 'Champions'
    elif row['RFM_Total'] >= 11:
        return 'Loyal Customers'
    elif row['RFM_Total'] >= 9:
        return 'Potential Loyalists'
    elif row['RFM_Total'] >= 7:
        return 'At Risk'
    elif row['RFM_Total'] >= 5:
        return 'Cannot Lose Them'
    else:
        return 'Lost Customers'

rfm['Segment'] = rfm.apply(rfm_segment, axis=1)

print("🎯 RFM ANALYSIS RESULTS")
print("=" * 50)
display(rfm.head(10))

# Segment distribution
segment_summary = rfm.groupby('Segment').agg({
    'Recency': 'mean',
    'Frequency': 'mean', 
    'Monetary': ['mean', 'sum'],
    'customer_id': 'count'
}).round(2)

segment_summary.columns = ['Avg_Recency', 'Avg_Frequency', 'Avg_Monetary', 'Total_Revenue', 'Customer_Count']

print("\n📊 Segment Summary:")
display(segment_summary)

In [ ]:
# Visualize RFM segments
segment_counts = rfm['Segment'].value_counts()

# Create subplots for RFM visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Customer Segments', 'RFM Score Distribution',
                   'Recency vs Frequency', 'Frequency vs Monetary'),
    specs=[[{"type": "pie"}, {"type": "histogram"}],
           [{"type": "scatter"}, {"type": "scatter"}]]
)

# Customer segments pie chart
fig.add_trace(
    go.Pie(labels=segment_counts.index, values=segment_counts.values, name='Segments'),
    row=1, col=1
)

# RFM total score distribution
fig.add_trace(
    go.Histogram(x=rfm['RFM_Total'], nbinsx=15, name='RFM Scores'),
    row=1, col=2
)

# Recency vs Frequency scatter
colors = {'Champions': 'gold', 'Loyal Customers': 'green', 'Potential Loyalists': 'blue',
          'At Risk': 'orange', 'Cannot Lose Them': 'red', 'Lost Customers': 'gray'}

for segment in rfm['Segment'].unique():
    segment_data = rfm[rfm['Segment'] == segment]
    fig.add_trace(
        go.Scatter(x=segment_data['Recency'], y=segment_data['Frequency'],
                  mode='markers', name=segment,
                  marker=dict(color=colors.get(segment, 'black'))),
        row=2, col=1
    )

# Frequency vs Monetary scatter
for segment in rfm['Segment'].unique():
    segment_data = rfm[rfm['Segment'] == segment]
    fig.add_trace(
        go.Scatter(x=segment_data['Frequency'], y=segment_data['Monetary'],
                  mode='markers', name=segment, showlegend=False,
                  marker=dict(color=colors.get(segment, 'black'))),
        row=2, col=2
    )

fig.update_layout(height=800, title_text="🎯 RFM Customer Segmentation Analysis")
fig.show()

print(f"\n📈 Total customers segmented: {len(rfm):,}")
print("\n🏆 Segment Recommendations:")
print("• Champions: Reward and retain them")
print("• Loyal Customers: Upsell premium products")
print("• Potential Loyalists: Develop loyalty programs")
print("• At Risk: Send personalized offers")
print("• Cannot Lose Them: Win-back campaigns")
print("• Lost Customers: Reactivation campaigns")

## 📦 Product Analysis

Phân tích hiệu suất sản phẩm và xu hướng

In [ ]:
# Product performance analysis
product_performance = transactions_df.groupby('product_id').agg({
    'quantity': 'sum',
    'total_amount': 'sum',
    'transaction_id': 'count',
    'customer_id': 'nunique'
}).round(2)

product_performance.columns = ['Total_Quantity', 'Total_Revenue', 'Total_Orders', 'Unique_Customers']

# Merge with product information
product_analysis = product_performance.merge(products_df, on='product_id', how='left')

# Calculate additional metrics
product_analysis['Revenue_per_Unit'] = product_analysis['Total_Revenue'] / product_analysis['Total_Quantity']
product_analysis['Customer_Penetration'] = (product_analysis['Unique_Customers'] / 
                                          transactions_df['customer_id'].nunique()) * 100

# Top products by different metrics
print("🏆 TOP PERFORMING PRODUCTS")
print("=" * 50)

print("\n💰 Top 10 by Revenue:")
top_revenue = product_analysis.nlargest(10, 'Total_Revenue')[['product_name', 'category', 'Total_Revenue', 'Total_Quantity']]
display(top_revenue)

print("\n📈 Top 10 by Quantity Sold:")
top_quantity = product_analysis.nlargest(10, 'Total_Quantity')[['product_name', 'category', 'Total_Quantity', 'Total_Revenue']]
display(top_quantity)

print("\n👥 Top 10 by Customer Reach:")
top_customers = product_analysis.nlargest(10, 'Unique_Customers')[['product_name', 'category', 'Unique_Customers', 'Customer_Penetration']]
display(top_customers)

In [ ]:
# Product category analysis
category_analysis = product_analysis.groupby('category').agg({
    'Total_Revenue': 'sum',
    'Total_Quantity': 'sum',
    'Total_Orders': 'sum',
    'Unique_Customers': 'sum',
    'product_id': 'count',
    'price': 'mean'
}).round(2)

category_analysis.columns = ['Total_Revenue', 'Total_Quantity', 'Total_Orders', 
                           'Total_Customers', 'Product_Count', 'Avg_Price']

category_analysis['Revenue_per_Product'] = category_analysis['Total_Revenue'] / category_analysis['Product_Count']
category_analysis = category_analysis.sort_values('Total_Revenue', ascending=False)

print("📊 CATEGORY PERFORMANCE ANALYSIS")
print("=" * 50)
display(category_analysis)

# Visualize category performance
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Revenue by Category', 'Average Price by Category',
                   'Products Count by Category', 'Revenue per Product'),
    specs=[[{"type": "bar"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "bar"}]]
)

# Revenue by category
fig.add_trace(
    go.Bar(x=category_analysis.index, y=category_analysis['Total_Revenue'],
           name='Revenue', marker_color='skyblue'),
    row=1, col=1
)

# Average price by category
fig.add_trace(
    go.Bar(x=category_analysis.index, y=category_analysis['Avg_Price'],
           name='Avg Price', marker_color='lightgreen'),
    row=1, col=2
)

# Product count by category
fig.add_trace(
    go.Bar(x=category_analysis.index, y=category_analysis['Product_Count'],
           name='Product Count', marker_color='salmon'),
    row=2, col=1
)

# Revenue per product
fig.add_trace(
    go.Bar(x=category_analysis.index, y=category_analysis['Revenue_per_Product'],
           name='Revenue per Product', marker_color='gold'),
    row=2, col=2
)

fig.update_layout(height=800, title_text="📦 Product Category Analysis")
fig.update_xaxes(tickangle=45)
fig.show()

## 🔍 Market Basket Analysis

Phân tích những sản phẩm thường được mua cùng nhau

In [ ]:
# Market basket analysis
from itertools import combinations
from collections import Counter

# Get transactions with multiple items
basket_transactions = transactions_df.groupby('transaction_id')['product_id'].apply(list).reset_index()
basket_transactions = basket_transactions[basket_transactions['product_id'].apply(len) > 1]

print(f"📊 Transactions with multiple items: {len(basket_transactions):,}")
print(f"📊 Total transactions: {len(transactions_df['transaction_id'].unique()):,}")
print(f"📊 Multi-item rate: {len(basket_transactions)/len(transactions_df['transaction_id'].unique())*100:.1f}%")

# Find frequent item pairs
item_pairs = []
for items in basket_transactions['product_id']:
    pairs = list(combinations(items, 2))
    item_pairs.extend(pairs)

# Count pair frequencies
pair_counts = Counter(item_pairs)
top_pairs = pair_counts.most_common(20)

# Create DataFrame for visualization
pair_df = pd.DataFrame(top_pairs, columns=['Product_Pair', 'Frequency'])
pair_df['Product_1'] = pair_df['Product_Pair'].apply(lambda x: x[0])
pair_df['Product_2'] = pair_df['Product_Pair'].apply(lambda x: x[1])

# Get product names
product_names = dict(zip(products_df['product_id'], products_df['product_name']))
pair_df['Product_1_Name'] = pair_df['Product_1'].map(product_names)
pair_df['Product_2_Name'] = pair_df['Product_2'].map(product_names)
pair_df['Pair_Name'] = pair_df['Product_1_Name'] + ' + ' + pair_df['Product_2_Name']

print("\n🛒 TOP 20 PRODUCT PAIRS (Frequently Bought Together)")
print("=" * 70)
display(pair_df[['Pair_Name', 'Frequency']].head(20))

# Visualize top pairs
fig = px.bar(pair_df.head(10), x='Frequency', y='Pair_Name',
             orientation='h', title='🛒 Top 10 Product Pairs (Market Basket)',
             labels={'Frequency': 'Co-occurrence Count', 'Pair_Name': 'Product Pairs'})
fig.update_layout(height=500, yaxis={'categoryorder': 'total ascending'})
fig.show()

## 🎯 Recommendation System Analysis

Phân tích để xây dựng hệ thống đề xuất sản phẩm

In [ ]:
# Create user-item interaction matrix
user_item_matrix = transactions_df.pivot_table(
    index='customer_id', 
    columns='product_id', 
    values='quantity', 
    aggfunc='sum', 
    fill_value=0
)

print(f"📊 User-Item Matrix Shape: {user_item_matrix.shape}")
print(f"📊 Sparsity: {(user_item_matrix == 0).sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100:.2f}%")

# Calculate basic recommendation metrics
# Most popular products (by number of unique customers)
product_popularity = transactions_df.groupby('product_id')['customer_id'].nunique().sort_values(ascending=False)
popular_products = product_popularity.head(20)

# Get product names for popular products
popular_products_df = popular_products.reset_index()
popular_products_df['product_name'] = popular_products_df['product_id'].map(product_names)
popular_products_df.columns = ['product_id', 'unique_customers', 'product_name']

print("\n⭐ TOP 20 MOST POPULAR PRODUCTS (by unique customers)")
print("=" * 60)
display(popular_products_df[['product_name', 'unique_customers']].head(20))

# Visualize popularity
fig = px.bar(popular_products_df.head(15), x='unique_customers', y='product_name',
             orientation='h', title='⭐ Top 15 Most Popular Products',
             labels={'unique_customers': 'Number of Unique Customers', 'product_name': 'Product'})
fig.update_layout(height=600, yaxis={'categoryorder': 'total ascending'})
fig.show()

In [ ]:
# Simple content-based recommendation example
def get_similar_products_by_category(product_id, top_n=5):
    """
    Get similar products based on category and price range
    """
    # Get target product info
    target_product = products_df[products_df['product_id'] == product_id]
    
    if target_product.empty:
        return []
    
    target_category = target_product['category'].iloc[0]
    target_price = target_product['price'].iloc[0]
    
    # Find similar products
    similar_products = products_df[
        (products_df['category'] == target_category) &
        (products_df['product_id'] != product_id) &
        (products_df['price'] >= target_price * 0.5) &
        (products_df['price'] <= target_price * 2)
    ]
    
    # Sort by price similarity
    similar_products['price_diff'] = abs(similar_products['price'] - target_price)
    similar_products = similar_products.sort_values('price_diff').head(top_n)
    
    return similar_products[['product_id', 'product_name', 'price']]

# Example: Get recommendations for a popular product
example_product_id = popular_products_df['product_id'].iloc[0]
example_product_name = popular_products_df['product_name'].iloc[0]

print(f"\n🎯 CONTENT-BASED RECOMMENDATIONS")
print(f"Target Product: {example_product_name} ({example_product_id})")
print("=" * 60)

recommendations = get_similar_products_by_category(example_product_id, 10)
display(recommendations)

# Customer purchase pattern analysis
customer_categories = transactions_df.groupby('customer_id')['category'].apply(list).reset_index()
customer_categories['unique_categories'] = customer_categories['category'].apply(lambda x: len(set(x)))
customer_categories['total_purchases'] = customer_categories['category'].apply(len)

# Category diversity analysis
diversity_stats = customer_categories['unique_categories'].describe()
print("\n📊 CUSTOMER CATEGORY DIVERSITY")
print("=" * 40)
print(f"Average categories per customer: {diversity_stats['mean']:.2f}")
print(f"Max categories per customer: {diversity_stats['max']:.0f}")
print(f"Customers buying from 1 category only: {(customer_categories['unique_categories'] == 1).sum():,} ({(customer_categories['unique_categories'] == 1).mean()*100:.1f}%)")

# Visualize category diversity
fig = px.histogram(customer_categories, x='unique_categories', nbins=20,
                   title='📊 Customer Category Diversity Distribution',
                   labels={'unique_categories': 'Number of Unique Categories per Customer',
                          'count': 'Number of Customers'})
fig.show()

## 📈 Business Intelligence & Insights

Tổng hợp các insights quan trọng cho doanh nghiệp

In [ ]:
# Key Business Metrics Dashboard
print("📊 KEY BUSINESS METRICS & INSIGHTS")
print("=" * 60)

# 1. Revenue Insights
total_revenue = transactions_df['total_amount'].sum()
avg_order_value = transactions_df['total_amount'].mean()
median_order_value = transactions_df['total_amount'].median()

print("\n💰 REVENUE INSIGHTS")
print(f"Total Revenue: {total_revenue:,.0f} VND")
print(f"Average Order Value: {avg_order_value:,.0f} VND")
print(f"Median Order Value: {median_order_value:,.0f} VND")

# 2. Customer Insights
total_customers = customers_df['customer_id'].nunique()
active_customers = transactions_df['customer_id'].nunique()
customer_activation_rate = active_customers / total_customers * 100

avg_orders_per_customer = transactions_df.groupby('customer_id').size().mean()
avg_revenue_per_customer = customer_behavior['total_spent'].mean()

print("\n👥 CUSTOMER INSIGHTS")
print(f"Total Registered Customers: {total_customers:,}")
print(f"Active Customers: {active_customers:,}")
print(f"Customer Activation Rate: {customer_activation_rate:.1f}%")
print(f"Average Orders per Customer: {avg_orders_per_customer:.1f}")
print(f"Average Revenue per Customer: {avg_revenue_per_customer:,.0f} VND")

# 3. Product Insights
total_products = products_df['product_id'].nunique()
products_sold = transactions_df['product_id'].nunique()
product_sell_through_rate = products_sold / total_products * 100

avg_price = products_df['price'].mean()
best_category = category_analysis.index[0]
best_category_revenue = category_analysis['Total_Revenue'].iloc[0]

print("\n📦 PRODUCT INSIGHTS")
print(f"Total Products in Catalog: {total_products:,}")
print(f"Products with Sales: {products_sold:,}")
print(f"Product Sell-through Rate: {product_sell_through_rate:.1f}%")
print(f"Average Product Price: {avg_price:,.0f} VND")
print(f"Best Performing Category: {best_category} ({best_category_revenue:,.0f} VND)")

# 4. Operational Insights
date_range = (transactions_df['timestamp'].max() - transactions_df['timestamp'].min()).days
daily_avg_transactions = len(transactions_df) / date_range
daily_avg_revenue = total_revenue / date_range

peak_hour = transactions_df.groupby('hour').size().idxmax()
peak_day = transactions_df['day_of_week'].value_counts().index[0]

print("\n⚡ OPERATIONAL INSIGHTS")
print(f"Data Period: {date_range} days")
print(f"Daily Average Transactions: {daily_avg_transactions:.1f}")
print(f"Daily Average Revenue: {daily_avg_revenue:,.0f} VND")
print(f"Peak Sales Hour: {peak_hour}:00")
print(f"Peak Sales Day: {peak_day}")

# 5. Recommendation System Potential
repeat_customers = customer_behavior[customer_behavior['num_orders'] > 1].shape[0]
repeat_rate = repeat_customers / len(customer_behavior) * 100

cross_category_customers = customer_categories[customer_categories['unique_categories'] > 1].shape[0]
cross_sell_potential = cross_category_customers / len(customer_categories) * 100

print("\n🎯 RECOMMENDATION SYSTEM POTENTIAL")
print(f"Repeat Customers: {repeat_customers:,} ({repeat_rate:.1f}%)")
print(f"Cross-category Customers: {cross_category_customers:,} ({cross_sell_potential:.1f}%)")
print(f"Multi-item Purchase Rate: {len(basket_transactions)/len(transactions_df['transaction_id'].unique())*100:.1f}%")

In [ ]:
# Final Summary Dashboard
print("\n" + "="*80)
print("🚀 HUIT BIG DATA PROJECT - EXECUTIVE SUMMARY")
print("="*80)

print("\n📋 PROJECT OVERVIEW:")
print("• E-commerce Data Analysis & Recommendation System")
print("• Big Data processing with Apache Spark")
print("• Machine Learning for product recommendations")
print("• Interactive web demo with analytics dashboard")

print("\n📊 DATA PROCESSED:")
print(f"• {len(customers_df):,} Customer records")
print(f"• {len(products_df):,} Product records")
print(f"• {len(transactions_df):,} Transaction records")
print(f"• {total_revenue:,.0f} VND total revenue analyzed")

print("\n🎯 KEY FINDINGS:")
print(f"• Customer activation rate: {customer_activation_rate:.1f}%")
print(f"• Average order value: {avg_order_value:,.0f} VND")
print(f"• Top category: {best_category}")
print(f"• Repeat purchase rate: {repeat_rate:.1f}%")
print(f"• Cross-sell potential: {cross_sell_potential:.1f}%")

print("\n🔮 RECOMMENDATION STRATEGIES:")
print("• Collaborative Filtering for similar customers")
print("• Content-based filtering for similar products")
print("• Market basket analysis for cross-selling")
print("• Customer segmentation for targeted marketing")
print("• Real-time personalization engine")

print("\n💡 BUSINESS RECOMMENDATIONS:")
print("• Focus on Champions and Loyal Customer segments")
print("• Implement win-back campaigns for At Risk customers")
print("• Optimize inventory for top-performing categories")
print("• Develop cross-selling strategies using basket analysis")
print("• Deploy recommendation system to increase AOV")

print("\n🌟 NEXT STEPS:")
print("• Deploy ML models to production")
print("• Set up real-time data streaming")
print("• A/B test recommendation algorithms")
print("• Monitor recommendation system performance")
print("• Scale system for larger datasets")

print("\n" + "="*80)
print("🎓 HUIT Big Data Analytics Project - Successfully Completed!")
print("="*80)

## 🎯 Conclusion

Notebook này đã thực hiện phân tích toàn diện dữ liệu mua sắm trực tuyến bao gồm:

### ✅ Các phân tích đã thực hiện:
1. **Exploratory Data Analysis (EDA)** - Khám phá dữ liệu cơ bản
2. **Customer Segmentation** - Phân khúc khách hàng bằng RFM Analysis
3. **Product Analysis** - Phân tích hiệu suất sản phẩm và danh mục
4. **Market Basket Analysis** - Tìm sản phẩm thường mua cùng nhau
5. **Recommendation System Foundation** - Nền tảng cho hệ thống đề xuất
6. **Business Intelligence** - Insights cho ra quyết định kinh doanh

### 🚀 Ứng dụng thực tế:
- Tối ưu hóa chiến lược marketing
- Cải thiện trải nghiệm khách hàng
- Tăng doanh số bán hàng
- Quản lý tồn kho hiệu quả
- Phát triển sản phẩm mới

### 📚 Kỹ thuật Big Data áp dụng:
- **Apache Spark** cho xử lý dữ liệu lớn
- **Machine Learning** cho recommendation systems
- **Data Visualization** cho business intelligence
- **Statistical Analysis** cho customer insights

---

**Tác giả:** Sinh viên HUIT  
**Môn học:** Big Data Analytics  
**Trường:** Đại học Công nghiệp TP.HCM